In [295]:
import pandas as pd

In [298]:
answer_key = pd.read_csv(r'annotator-assessment-profiler\answer\answer_key.csv')
responses = pd.read_csv(r'C:\Users\jiayue.tan\Downloads\YTL AI Labs Annotator Application & Assessment(Applicants) (1).csv',header=[0,1])

In [299]:
# Create a new, single-level list of column names by joining the two levels
# Example: ('Malay Q1 (Prompt)', 'Evaluate whether...') -> 'Malay Q1 (Prompt) | Evaluate whether...'
responses.columns = [' | '.join(col).strip() for col in responses.columns.values]

In [300]:
all_columns = responses.columns.tolist()
prefixes = ("Malay", "English", "Chinese")

# 3. Use list comprehension to filter the columns
language_columns = [
    col for col in all_columns
    if col.startswith(prefixes)
]

In [301]:
import re

# Define the full text patterns you want to hard-rename
PROMPT_STATUS_PATTERN = "evaluate whether the prompt.*is usable.*"
RESPONSE_STATUS_PATTERN = "evaluate the quality of the response.*in the bolded blue text below.*"

# The dictionary to store the final mapping
column_map = {}

for old_col in responses.columns:
    # 1. Lowercase the entire column for consistent matching
    lower_col = old_col.lower().strip()

    # 2. Split the column into the prefix and the evaluation question
    # Example: 'malay q1 (prompt) | evaluate...'
    parts = lower_col.split(' | ', 1)
    if len(parts) < 2:
        # Skip columns that don't match the expected MultiIndex format
        continue

    prefix, question = parts[0], parts[1]

    # Clean up the prefix: 'malay q1 (prompt)' -> 'malay_q1_prompt'
    # Use re.sub to remove non-alphanumeric/non-space characters, then replace spaces with underscores
    cleaned_prefix = re.sub(r'[^a-z0-9\s]', '', prefix).replace(' ', '_').strip('_')
    
    # 3. Apply the mapping logic to determine the suffix
    new_suffix = None
    if re.search(PROMPT_STATUS_PATTERN, question):
        new_suffix = "status"
    elif re.search(RESPONSE_STATUS_PATTERN, question):
        new_suffix = "status"
    
    # If a new suffix was found, construct the new column name
    if new_suffix:
        new_col_name = f"{cleaned_prefix}_{new_suffix}"
        
        # 4. Add the original column (case-sensitive) and the new name to the map
        column_map[old_col] = new_col_name
    
# Print the final mapping dictionary
print("### Generated Column Rename Map ###")
print("column_map = {")
for old, new in column_map.items():
    print(f'    "{old}": "{new}",')
print("}")

# Now you can apply this to your DataFrame:
responses.rename(columns=column_map, inplace=True)

### Generated Column Rename Map ###
column_map = {
    "Malay Q1 (Prompt) | Evaluate whether the prompt in the image above OR in the bolded blue text below is usable.": "malay_q1_prompt_status",
    "Malay Q1 (Response) | Evaluate the quality of the response in the image above OR in the bolded blue text below.": "malay_q1_response_status",
    "Malay Q2 (prompt) | Evaluate whether the prompt written in the image above OR in the bolded blue text below is usable.": "malay_q2_prompt_status",
    "Malay Q2 (response) | Evaluate the quality of the response written in the image above OR in the bolded blue text below.": "malay_q2_response_status",
    "Malay Q3 (prompt) | Evaluate whether the prompt written in the image above OR in the bolded blue text below is usable.": "malay_q3_prompt_status",
    "Malay Q3 (response) | Evaluate the quality of the response written in the image above OR in the bolded blue text below": "malay_q3_response_status",
    "Malay Q4 (prompt) | Evaluate whether the

In [302]:
import numpy as np

# 1. Define the specific hard rename map for the CONFIDENCE_COLUMNS
CONFIDENT_LANGUAGE_MAP = {
    'Unnamed: 15_level_0 | This question is about Bahasa Melayu (BM) language abilities.\nAre you confident reviewing and writing in both everyday/casual language (e.g. social media posts, chat messages) and more formal language': 'malay_confidence',
    'Unnamed: 16_level_0 | This question is about English language abilities.\nAre you confident reviewing and writing in both everyday/casual language (e.g. social media posts, chat messages) and more formal language (e.g. news': 'english_confidence',
    'Unnamed: 17_level_0 | This question is about Mandarin Chinese language abilities.\nAre you comfortable reviewing and writing in both everyday/casual language (e.g. social media posts, chat messages) and more formal language': 'chinese_confidence'
}

# 2. Define the function for value encoding
def encode_confidence(value):
    """Encodes the string value to 0 if 'no' is present, or 1 otherwise."""
    if pd.isna(value):
        return np.nan
    # Convert to string and check if 'no' is in the lowercased value
    if 'no' in str(value).lower():
        return 0
    return 1

# 3. Define the main function to load, rename, and process the data
def process_confidence_columns(df):

    df.rename(columns=CONFIDENT_LANGUAGE_MAP, inplace=True)

    new_confidence_cols = list(CONFIDENT_LANGUAGE_MAP.values())

    for col in new_confidence_cols:
        if col in df.columns:
            df[col] = df[col].apply(encode_confidence)

    print(df[new_confidence_cols].head())
    print(df[new_confidence_cols].info())
    
    return df

responses2 = process_confidence_columns(responses)

   malay_confidence  english_confidence  chinese_confidence
0                 1                   1                   0
1                 1                   1                   1
2                 1                   1                   1
3                 1                   1                   1
4                 1                   1                   1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   malay_confidence    203 non-null    int64
 1   english_confidence  203 non-null    int64
 2   chinese_confidence  203 non-null    int64
dtypes: int64(3)
memory usage: 4.9 KB
None


In [303]:
ID_COLUMNS_MAP = {'Unnamed: 6_level_0 | Full name (according to NRIC):':"name",
                  'Unnamed: 9_level_0 | IC number:':"ic_number"}

responses2.rename(columns=ID_COLUMNS_MAP,inplace=True)

responses2.head(2)

responses3 = responses2.copy()

In [304]:
renamed_scoring_columns = [
    col for col in responses3.columns
    if col.startswith(("malay_","chinese_","english_"))
]

scoring_columns = ['ic_number','name']+ renamed_scoring_columns
responses3 = responses3[scoring_columns]
responses3.head(2)

,ic_number,name,malay_confidence,english_confidence,chinese_confidence,malay_q1_prompt_status,malay_q1_response_status,malay_q2_prompt_status,malay_q2_response_status,malay_q3_prompt_status,...,chinese_q1_prompt_status,chinese_q1_response_status,chinese_q2_prompt_status,chinese_q2_response_status,chinese_q3_prompt_status,chinese_q3_response_status,chinese_q4_prompt_status,chinese_q4_response_status,chinese_q5_prompt_status,chinese_q5_response_status
0,60511081143,Tang Eugine,1,1,0,"FAIL (Minor rewrite, ≤ 2 sentences)",PASS (no changes),PASS,PASS (no changes),PASS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50111080310,Tang Yvone,1,1,1,"FAIL (Major rewrite, > 2 sentences)",PASS (no changes),PASS,FAIL (needs improvement),PASS,...,"FAIL (Minor Rewrite, ≤ 2 sentences)",PASS (no changes),PASS,FAIL (needs improvement),PASS,PASS (no changes),PASS,FAIL (needs improvement),PASS,PASS (no changes)


In [305]:
responses3.columns

Index(['ic_number', 'name', 'malay_confidence', 'english_confidence',
       'chinese_confidence', 'malay_q1_prompt_status',
       'malay_q1_response_status', 'malay_q2_prompt_status',
       'malay_q2_response_status', 'malay_q3_prompt_status',
       'malay_q3_response_status', 'malay_q4_prompt_status',
       'malay_q4_response_status', 'malay_q5_prompt_status',
       'malay_q5_response_status', 'english_q1_prompt_status',
       'english_q1_response_status', 'english_q2_prompt_status',
       'english_q2_response_status', 'english_q3_prompt_status',
       'english_q3_response_status', 'english_q4_prompt_status',
       'english_q4_response_status', 'english_q5_prompt_status',
       'english_q5_response_status', 'chinese_q1_prompt_status',
       'chinese_q1_response_status', 'chinese_q2_prompt_status',
       'chinese_q2_response_status', 'chinese_q3_prompt_status',
       'chinese_q3_response_status', 'chinese_q4_prompt_status',
       'chinese_q4_response_status', 'chinese_q5

In [306]:
import pandas as pd
import numpy as np
import re

# --- Core Scoring Functions ---

def clean_answer(ans):
    """
    Standardizes the answer string to 'PASS' or 'FAIL' if found, 
    otherwise returns None (for NaN/missing/unmatched strings).
    """
    if pd.isna(ans) or ans is None:
        return None
    
    ans_str = str(ans).upper().strip()
    
    if re.search(r'PASS', ans_str):
        return 'PASS'
    elif re.search(r'FAIL', ans_str):
        return 'FAIL'
        
    return None

def score_match(response_ans, key_ans):
    """
    Applies the custom scoring logic for a single question response:
    1. key=FAIL and response=PASS => 0.0
    2. key=PASS and response=FAIL => 0.5
    3. Matched => 1.0
    4. NaN in key or response => NaN
    """
    clean_response = clean_answer(response_ans)
    clean_key = clean_answer(key_ans)

    # Rule 4: Cannot score if either is missing or unreadable
    if clean_response is None or clean_key is None:
        return np.nan 

    # Rule 3: Perfect match
    if clean_key == clean_response:
        return 1.0 
    
    # Mismatched Cases
    if clean_key == 'FAIL' and clean_response == 'PASS':
        # Rule 1: Key says FAIL, Response says PASS => 0.0
        return 0.0
    elif clean_key == 'PASS' and clean_response == 'FAIL':
        # Rule 2: Key says PASS, Response says FAIL => 0.5
        return 0.5
    
    return np.nan

# --- Narrative Summary Function (Revised for Integration) ---

def generate_narrative_summary(responses_df, answer_key_df):
    """
    Creates a multi-line narrative summary of scoring mismatches for each respondent.
    This function MODIFIES responses_df by adding the narrative columns.
    """
    languages = ['malay', 'english', 'chinese']
    
    # 1. Prepare Answer Key Lookup (Map question_key to original answer)
    key_map = answer_key_df.set_index('question_key')['answer'].to_dict()
    
    # 2. Iterate row-by-row (per respondent) to build the narrative
    # Use .apply() for better performance than iterrows()
    
    def create_narrative(row, lang):
        lang_narrative_list = []
        
        # Get score columns for the specific language (e.g., 'malay_q1_prompt_status_score')
        q_score_cols = [col for col in row.index if col.startswith(f'{lang}_') and col.endswith('_score')]
        
        lang_total_score = row.get(f'{lang}_scores', 0.0) # Get the final calculated score
        
        # Check for any scored questions
        if not q_score_cols:
             return f"Total Score: 0.0. No scorable questions for {lang}."

        # Iterate over the individual question scores
        for score_col in q_score_cols:
            score = row.get(score_col)
            
            # Only focus on partial (0.5) or zero (0.0) scores
            if score == 0.5 or score == 0.0:
                question_key = score_col.replace('_score', '')
                
                # Look up the original key and response answers
                key_ans = key_map.get(question_key, 'KEY_NOT_FOUND')
                response_ans = row.get(question_key, 'RESPONSE_NOT_FOUND')
                
                # Clean answers for the explanation
                clean_key = clean_answer(key_ans)
                clean_response = clean_answer(response_ans)

                # Construct the detailed explanation
                explanation = f"Key: {clean_key} (Original: '{key_ans}'), Response: {clean_response} (Original: '{response_ans}')"
                
                lang_narrative_list.append(
                    f"{question_key} => Score: {score} => Mismatch: {explanation}"
                )

        # 3. Compile the final narrative for the language
        if lang_narrative_list:
            header = f"Total Score: {lang_total_score} out of {len(q_score_cols)} questions. Mismatches ({len(lang_narrative_list)}):"
            return header + "\n" + "\n".join(lang_narrative_list)
        else:
            return f"Total Score: {lang_total_score} out of {len(q_score_cols)} questions. (All Perfect Matches - Score 1.0)"


    # 4. Apply the narrative creation function to each row
    for lang in languages:
        responses_df[f'{lang}_narrative'] = responses_df.apply(
            lambda row: create_narrative(row, lang), axis=1
        )
        
    # Return the columns with the final narratives
    return responses_df[['ic_number', 'name', 'malay_narrative', 'english_narrative', 'chinese_narrative']]


# --- Main Score Calculation Function ---

def calculate_all_scores(responses_df, answer_key_df):
    """Calculates individual question scores, sums them, and generates the narrative summary."""
    
    # 1. Prepare Answer Key for Scoring
    answer_key_df['clean_answer'] = answer_key_df['answer'].apply(clean_answer)

    # 2. Calculate Individual Question Scores
    for _, row in answer_key_df.iterrows():
        q_key = row['question_key']
        clean_key = row['clean_answer']
        
        if q_key in responses_df.columns:
            score_col_name = f"{q_key}_score"
            responses_df[score_col_name] = responses_df[q_key].apply(
                lambda resp_ans: score_match(resp_ans, clean_key)
            )

    # 3. Calculate Total Language Scores
    lang_prefixes = ['malay_', 'english_', 'chinese_']
    final_score_cols = []
    
    for prefix in lang_prefixes:
        all_score_cols = [col for col in responses_df.columns if col.endswith('_score')]
        lang_score_cols = [col for col in all_score_cols if col.startswith(prefix)]
        new_score_col_name = f'{prefix}scores'
        
        if lang_score_cols:
            responses_df[new_score_col_name] = responses_df[lang_score_cols].sum(axis=1)
        else:
            responses_df[new_score_col_name] = 0.0 
            
        final_score_cols.append(new_score_col_name)

    # 4. Generate Narrative Summary (The New Step)
    narrative_df = generate_narrative_summary(responses_df, answer_key_df)

    # 5. Combine and Return Final Output
    # Merge the total scores and confidence columns with the new narrative columns
    output_df = responses_df[[
        'ic_number', 'name', 
        'malay_confidence', 'english_confidence', 'chinese_confidence'
    ] + final_score_cols].merge(
        narrative_df, on=['ic_number', 'name'], how='left'
    )
    
    return output_df

# --- EXECUTION ---
# FINAL STEP: You need to replace this block with your actual loaded DataFrames:
# # Assuming you have loaded your data like this:
responses_df = responses3 
answer_key_df = answer_key

final_output_df = calculate_all_scores(responses_df, answer_key_df)
final_output_df.head()

,ic_number,name,malay_confidence,english_confidence,chinese_confidence,malay_scores,english_scores,chinese_scores,malay_narrative,english_narrative,chinese_narrative
0,60511081143,Tang Eugine,1,1,0,5.0,5.0,0.0,Total Score: 5.0 out of 10 questions. Mismatch...,Total Score: 5.0 out of 10 questions. Mismatch...,Total Score: 0.0 out of 10 questions. (All Per...
1,50111080310,Tang Yvone,1,1,1,7.0,6.0,7.0,Total Score: 7.0 out of 10 questions. Mismatch...,Total Score: 6.0 out of 10 questions. Mismatch...,Total Score: 7.0 out of 10 questions. Mismatch...
2,50625140990,KHONG YIROU,1,1,1,6.5,5.5,6.5,Total Score: 6.5 out of 10 questions. Mismatch...,Total Score: 5.5 out of 10 questions. Mismatch...,Total Score: 6.5 out of 10 questions. Mismatch...
3,50609050370,SIA ZI YUE,1,1,1,7.0,8.0,5.0,Total Score: 7.0 out of 10 questions. Mismatch...,Total Score: 8.0 out of 10 questions. Mismatch...,Total Score: 5.0 out of 10 questions. Mismatch...
4,50211010344,OI KAY YI,1,1,1,6.0,9.0,7.5,Total Score: 6.0 out of 10 questions. Mismatch...,Total Score: 9.0 out of 10 questions. (All Per...,Total Score: 7.5 out of 10 questions. Mismatch...


In [68]:
final_output_df.to_csv(r'C:\Users\jiayue.tan\OneDrive - YTL\Workstation\dev\annotator-assessment-profiler\output.csv')